In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import random
warnings.filterwarnings('ignore')

In [ ]:
%pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 54.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=45116c0e1770773c43f8ee9c3211f29ac11d3a91d88174e2bdc874319083d712
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [ ]:
df_lyrics = pd.read_csv('lyrics_10k.csv')
df_lyrics.sample(5)

,Unnamed: 0,song_id,artist_id,song,artists,explicit,genres,lyrics
1988,1455,spotify:track:7EmvKUGH8z4c9NyQ9pADUn,spotify:artist:5mYWkDD4b1eM4ZjFq5axxs,Emotions,['Aidonia'],False,dancehall;reggae fusion;riddim;soca;traphall,Emotions Lyrics When we met yuh seh n'ave no m...
9184,8651,spotify:track:3ozVz5TabkIiWtDZi3sFEQ,spotify:artist:5WUlDfRSoLAfcVSX1WnrxN,Footprints,['Sia'],False,australian dance;australian pop;dance pop;pop,Translations Français Footprints Lyrics Oh oh...
8257,7724,spotify:track:6pUAg3OgdO2kQ4lub9Tb5m,spotify:artist:2cCUtGK9sDU2EoElnk0GNB,Terrible Love,['The National'],False,indie rock;modern rock;stomp and holler,Terrible Love Lyrics It's a terrible love and ...
5393,4860,spotify:track:79ilggJGPbtHVhNzVob5ox,spotify:artist:26VFTg2z8YR0cCuwLzESi2,Castle,['Halsey'],False,dance pop;electropop;etherpop;indie poptimism;pop,Translations Español Italiano Français Dansk C...
8824,8291,spotify:track:2CEa1JmOQlC1NndASOwssY,spotify:artist:3CygdxquGHurS7f9LjNLkv,Good Lord Willing,['Little Big Town'],False,contemporary country;country;country dawn;coun...,Good Lord Willing Lyrics Long about midnight U...


In [ ]:
df_lyrics.shape

(9207, 8)

In [ ]:
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0  # for consistent results

def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

df_lyrics['is_english'] = df_lyrics['lyrics'].apply(is_english)
df_lyrics = df_lyrics[df_lyrics['is_english']]

In [ ]:
df_lyrics.shape

(8689, 9)

In [ ]:
import re
import string

def preprocess_lyrics(text):
    # Lowercase
    text = text.lower()
    # Remove translations headers like "Translations Español", "Lyrics"
    text = re.sub(r"translations\s+\w+|lyrics", "", text, flags=re.IGNORECASE)
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove non-alphabetic characters
    text = re.sub(r"[^a-z\s]", "", text)
    # Remove extra whitespace
    text = re.sub(r"\s+", " ", text).strip()
    return text

df_lyrics['clean_lyrics'] = df_lyrics['lyrics'].apply(preprocess_lyrics)

In [ ]:
df_lyrics = df_lyrics.drop_duplicates(subset='clean_lyrics')
df_lyrics = df_lyrics.dropna(subset=['clean_lyrics'])

In [ ]:
df_lyrics.shape

(6458, 10)

In [ ]:
df_lyrics.isnull().sum()

,0
Unnamed: 0,0
song_id,0
artist_id,0
song,0
artists,0
explicit,0
genres,56
lyrics,0
is_english,0
clean_lyrics,0


In [ ]:
df_lyrics.dropna(inplace=True)

In [ ]:
df_lyrics.reset_index(drop=True, inplace=True)

In [ ]:
import re

df_lyrics['song_id'] = df_lyrics['song_id'].apply(lambda x: re.sub(r'spotify:track:', '', x))
df_lyrics.sample(5)

,Unnamed: 0,song_id,artist_id,song,artists,explicit,genres,lyrics,is_english,clean_lyrics
5636,7406,7n2qWHQ6l7LnHQ5KC6mUJj,spotify:artist:50co4Is1HCEo8bhOyUWKpn,Eww,"['Young Thug', {'label': 'spotify:artist:5lHRU...",True,atl hip hop;atl trap;gangster rap;hip hop;melo...,Eww Lyrics Eww eww eww Eww eww eww I'm hig...,True,eww eww eww eww eww eww eww im high right now ...
2906,2934,5HkfjD9X8AHBm0I02zjoYF,spotify:artist:55Aa2cqylxrFIXC767Z865,Spitter,['Lil Wayne'],False,hip hop;new orleans rap;pop rap;rap;trap,Spitter Lyrics Spit yo' game talk yo' shit Gr...,True,spitter spit yo game talk yo shit grab yo gat ...
2153,1966,0AJZnHScoJjMFpytRsqdnf,spotify:artist:0f8MDDzIc6M4uH1xH0o0gy,When You Got a Good Friend,['Robert Johnson'],False,acoustic blues;blues;delta blues;pre-war blues...,When You Got a Good Friend Lyrics When you go...,True,when you got a good friend when you got a good...
5558,6756,4q9x6YrGoeSh8NO55ikmL7,spotify:artist:7hJcb9fa4alzcOq3EaNPoG,We Rest N Cali,"['Snoop Dogg', {'label': 'spotify:artist:5K0rb...",False,g funk;gangster rap;hip hop;pop rap;rap;west c...,We Rest N Cali Lyrics My goodness my gracious ...,True,we rest n cali my goodness my gracious shell t...
2965,3015,659HIjo4YGCavNiVEgqBtI,spotify:artist:3JFsVIxOn7STeilPICkkB2,The Undertow,['Lamb of God'],False,alternative metal;groove metal;melodic metalco...,The Undertow Lyrics You always needed people l...,True,the undertow you always needed people like me ...


In [ ]:
df_lyrics['lyrics'][0]

"Better than home Lyrics I can feel my body breathing I can feel my heart is moving fast I am not afraid or lonely I am not chasing the ghosts of the past I have found the place where hunger Meets the edge and now I'm facing God I won't dare look into his eyes I can only hang my head to the ground When I try and open my mouth There's so many words but there's no sound And the angel comes upon me And the love I feel's a love I've never known And it's better than home Better than home Out on this long and winding road Chasing the sound with my friends And we ain't never rolling back again And who knows what we're gonna find Knows what we're gonna see Knows how we're gonna change Knows what we're gonna be Baby it's just you and me So keep driving on Cause this is better than home Everything inside me is stirring In every corner of every secret wound In the light I am adoring Feel like flying in circles around the moon When I gaze upon the world I know that I'm a part of something good I k

In [ ]:
df_lyrics['lyrics'][9]

"Slide Lyrics Even on a day like this When you're crawling on the floor Reaching for the phone to ring Anyone who knows you anymore It's all right to make mistakes  you're only human Inside everybody's hiding something Staring at the same four walls Have you tried to help yourself?Rings around your eyes  they don't hide That you need to get some rest It's all right to make mistakes  you're only human Inside everybody's hiding something Take time to catch your breath and choose your moment Don't slide Don't slide  don't slide Don't slide  slide  slide Even at a time like this When the morning seems so far Think that pain belongs to you But it's happened to us all You bought this on yourself And it's high time you left it there Lie here and rest your head Dream of something else instead You might also like Embed"

In [ ]:
%pip install -q transformers accelerate bitsandbytes torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 130.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.0 MB/s eta 0:00:00


In [ ]:
!pip install -U bitsandbytes

In [ ]:
from google.colab import userdata
my_secret_key = userdata.get('huggingface_key')

In [ ]:
from huggingface_hub import login
login(token=userdata.get('huggingface_key'))

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

In [ ]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# Set up 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# Load tokenizer and quantized model
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",          # Automatically offloads to GPU
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True      # If the model has custom code
)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [ ]:
model.save_pretrained('llama_model')

In [ ]:
tokenizer.save_pretrained('llama_tokenizer')

('llama_tokenizer/tokenizer_config.json',
 'llama_tokenizer/special_tokens_map.json',
 'llama_tokenizer/chat_template.jinja',
 'llama_tokenizer/tokenizer.json')

In [ ]:
model = AutoModelForCausalLM.from_pretrained('llama_model')
tokenizer = AutoTokenizer.from_pretrained('llama_tokenizer')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
text_gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    device_map="auto"           # Ensures text_gen uses GPU
)

Device set to use cuda:0


In [ ]:
import pandas as pd
from tqdm import tqdm
import re

# Step 1: Take a sample of 500
df_sample = df_lyrics.sample(500, random_state=42).reset_index(drop=True)

In [ ]:
def make_prompt(lyrics_text):
    return [
        {"role": "system", "content": "You are a music mood classifier."},
        {"role": "user", "content": f"""Classify the following song lyrics into **one** of the following moods:
- Relaxation and stress relief
- Uplifting and motivational
- Social gatherings or parties

Only return the most suitable mood label from the list above, enclosed in [ANS] and [/ANS] tags. Do not explain.

Lyrics:
{lyrics_text}
"""}
    ]


# Step 3: Extract mood from output

valid_moods = [
    "Relaxation and stress relief",
    "Uplifting and motivational",
    "Social gatherings or parties"
]

def extract_mood(output_text):
    # Step 1: If model used [ANS] tags, extract from that
    ans_match = re.search(r"\[ANS\](.*?)\[/ANS\]", output_text, re.DOTALL)
    if ans_match:
        candidate = ans_match.group(1).strip()
        if candidate in valid_moods:
            return candidate

    # Step 2: Split after the last appearance of the lyrics section
    # Look for the last closing tag or marker
    split_marker = 'Lyrics:'  # Change this if you alter the prompt format
    parts = output_text.split(split_marker)
    if len(parts) > 1:
        # Get the part *after* the lyrics block
        post_lyrics = parts[-1]
        # Try to match valid mood from that section only
        for mood in valid_moods:
            if mood.lower() in post_lyrics.lower():
                return mood

    # Step 3: Fallback if nothing found
    return "unknown"


# Step 4: Generate moods using your LLaMA model
predicted_moods = []

for lyrics in tqdm(df_sample['lyrics']):
    prompt = make_prompt(lyrics[:2000])  # clip long lyrics if needed
    try:
        output = text_gen(prompt, max_new_tokens=100, do_sample=False)[0]['generated_text']
        mood = extract_mood(output)
    except Exception as e:
        mood = "error"
    predicted_moods.append(mood)

# Step 5: Save predictions
df_sample['predicted_mood'] = predicted_moods

100%|██████████| 500/500 [09:14<00:00,  1.11s/it]


In [ ]:
df_sample['predicted_mood'].value_counts()

,count
predicted_mood,
Relaxation and stress relief,381
Uplifting and motivational,109
Social gatherings or parties,10


In [ ]:
for lyrics in df_sample['lyrics'][:10]:
    prompt = make_prompt(lyrics[:2000])
    output = text_gen(prompt)[0]['generated_text']
    if extract_mood(output) == "Uplifting and motivational":
        print("===================Start====================")
        print(output)
        print("===================End====================")

===================Start====================
[ANS] Uplifting and motivational [/ANS]
===================End====================


# Using Mistral

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch

model_mistral = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer_mistral = AutoTokenizer.from_pretrained(model_mistral, use_fast=True)
tokenizer_mistral.pad_token = tokenizer_mistral.eos_token

model_mistral = AutoModelForCausalLM.from_pretrained(
    model_mistral,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

text_gen_mistral = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer_mistral,
    return_full_text=False,
    device_map="auto"
)


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
def make_prompt_mistral(lyrics_text):
    return f"""You are a music mood classifier.

Classify the following song lyrics into **one** of the following moods:
- Relaxation and stress relief
- Uplifting and motivational
- Social gatherings or parties
- Sadness or melancholy

Only return the most suitable mood label from the list above, enclosed in [ANS] and [/ANS] tags. Do not explain.

Lyrics:
{lyrics_text}
"""

In [ ]:
def extract_mood_mistral(output_text):
    import re
    match = re.search(r"\[ANS\](.*?)\[/ANS\]", output_text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return "unknown"

from tqdm import tqdm

predicted_moods = []
for lyrics in tqdm(df_sample['lyrics']):
    prompt = make_prompt_mistral(lyrics[:2000])  # Clip to avoid overflow
    try:
        output = text_gen(prompt, max_new_tokens=100, do_sample=False)[0]['generated_text']
        mood = extract_mood_mistral(output)
    except Exception as e:
        mood = "error"
    predicted_moods.append(mood)

df_sample["predicted_mood_mistral"] = predicted_moods

100%|██████████| 500/500 [1:16:10<00:00,  9.14s/it]


In [ ]:
df_sample['predicted_mood_mistral'].value_counts()

,count
predicted_mood_mistral,
Uplifting and motivational,214
Sadness or melancholy,133
Relaxation and stress relief,121
unknown,18
Social gatherings or parties,14


In [ ]:
(df_sample['predicted_mood'] == df_sample['predicted_mood_mistral']).mean()

np.float64(0.43)

In [ ]:
df_sample = df_sample[df_sample['predicted_mood_mistral'] != 'unknown']
df_sample = df_sample.reset_index(drop=True)

In [ ]:
df_sample.to_csv('predicted_moods.csv', index=False)